## Every Dream trainer

You will need your data prepared first before starting!  Don't waste rental fees if you're not ready to upload your files.  Your files should be captioned before you start with either the caption as the filename or in text files for each image alongside the image files.  See main README.md for more details. Tools are available to automatically caption your files.

[Instructions](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md)

If you can sign up for Runpod here (shameless referral link): [Runpod](https://runpod.io?ref=oko38cd0)

If you are confused by the wall of text, join the discord here: [EveryDream Discord](https://discord.gg/uheqxU6sXN)

Make sure you have at least 40GB of Runpod **Volume** storage at a minimum so you don't waste training just 1 ckpt that is overtrained and have to start over.  Penny pinching on storage is ultimately a waste of your time and money!  This is setup to give you more than one ckpt so you don't overtrain.

### Starting model
Make sure you have your hugging face token ready to download the 1.5 mode. You can get one here: https://huggingface.co/settings/tokens
If you don't have a User Access Token, create one.  Or you can upload a starting checkpoint instead of using the HF download and skip that step, but you'll need to modify the starting model name when you start training (more info below).

In [ ]:
I_LIKE_FUN_STUFF = True

!git clone https://github.com/victorchall/everydream-trainer
%cd everydream-trainer

import scripts.fun
from IPython.display import Audio
finish_msg = codecs.encode('QBAR', 'rot_13')
sound_file = 'demo/wow.mp3'

print(finish_msg)

## Install dependencies

This will take a couple minutes. You can ignore "warnings."

In [ ]:
!pip install -q omegaconf
!pip install -q einops
!pip install -q pytorch-lightning==1.6.5
!pip install -q test-tube
!pip install -q transformers==4.19.2
!pip install -q kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -q setuptools==59.5.0
!pip install -q pillow==9.0.1
!pip install -q torchmetrics==0.6.0
!pip install -e .
#!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
import ipywidgets as widgets
print(finish_msg)
Audio(sound_file, autoplay=I_LIKE_FUN_STUFF)

## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your huggingface account page.

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="runwayml/stable-diffusion-v1-5",
 filename="v1-5-pruned.ckpt",
 use_auth_token=True
)
print(finish_msg)

# Upload training files

Ues the navigation on the left to open **"every-dream-trainer"** folder, then open **"input"** and upload your training files using the **up arrow button** above the file explorer.  

You can check there are files in the folder by running the cell below (optional, just prints first 10).

In [ ]:
!ls -U input | head -10
# at least a few image filenames should show below, if not you uploaded to the wrong folder

## Tweak your YAML
You can adjust the YAML file to change the training parameters.  The v1-fine yamls are in the everydream-trainer/configs/stable-diffusion folder. 

You can also upload your own in that folder and use it below.

Instructions are here: https://github.com/victorchall/EveryDream-trainer/blob/main/README.md (hopefully you already read this?)

[Runpod YAML](everydream-trainer/configs/stable-diffusion/v1-finetune_runpod.yaml) is a good starting point for small datasets (30-50 images) and is the default in the command below. It will only keep 2 checkpoints.

[EveryDream YAML](workspace/everydream-trainer/configs/stable-diffusion/v1-finetune_everydream.yaml) is a good starting point for large datasets. You will need to change the filename in the --config parameter below to use this.  This may create a LOT of large ckpt files while training, so make sure you have enough space in your runpod instance!  60GB+ is recommended.

# Run the trainer
This will take a while.  Make sure when it finishes you scroll down to run the last cell.

In [ ]:
# change to use a custom yaml, but run this cell no matter what
my_yaml = "configs/stable-diffusion/v1-finetune_runpod.yaml"

In [1]:
# run the trainer, wait until it finishes then SCROLL DOWN to the next cell
!python main.py --base {my_yaml} -t  --actual_resume "v1-5-pruned.ckpt" -n test --data_root input
Audio(sound_file, autoplay=I_LIKE_FUN_STUFF)
print(finish_msg)

Running on GPUs 0,
Loading model from v1-5-pruned.ckpt
Instantiating config for: ldm.models.diffusion.ddpm.LatentDiffusion with config:
{'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'image', 'cond_stage_key': 'caption', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': True, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'unfreeze_model': True, 'model_lr': 1e-06, 'unet_config': {'target': 'ldm.modules.diffusionmodules.openaimodel.UNetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'first_stage_config': {'target': 'ldm.models.autoencoder.AutoencoderKL', 'params': {'emb

Global seed set to 23
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.mlp.fc2.bias', 'vision_model.encoder.layers.0.self_attn.v_proj.weight', 'visual_projection.weight', 'vision_model.encoder.layers.14.mlp.fc1.bias', 'vision_model.encoder.layers.2.self_attn.out_proj.bias', 'vision_model.encoder.layers.15.self_attn.q_proj.weight', 'vision_model.encoder.layers.13.layer_norm2.weight', 'vision_model.encoder.layers.18.mlp.fc1.weight', 'vision_model.encoder.layers.15.mlp.fc2.weight', 'vision_model.encoder.layers.20.self_attn.k_proj.weight', 'vision_model.encoder.layers.22.self_attn.k_proj.weight', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.14.layer_norm2.bias', 'vision_model.encoder.layers.6.self_attn.out_proj.weight', 'vision_model.encoder.layers.14.self_attn.out_proj.bias', 'vision_model.encoder.layers.17.mlp.fc2.bias', 'vision_model.encoder.layers.17.la

NameError: name 'Audio' is not defined

## Prune your checkpoints
This will create 2GB pruned files for all your checkpoints. 

In [ ]:
# prune the ckpts
!python auto_prune_all.py --delete

## Download your checkpoints

Use the file explorer on the left, go into the "every-dream-trainer" folder.

Look for all the ckpt files that say "-pruned" on the end.  Download them and you're done! 

[EveryDream Discord](https://discord.gg/uheqxU6sXN)